# Load Module and data

In [1]:
from posixpath import join
from astropy.io.fits import hdu
import pandas as pd 
from matplotlib import pyplot as plt 
import numpy as np 
from ciao_contrib.runtool import search_csc 

from astropy.io import fits 
from fields import fields 
from tqdm import tqdm

In [7]:
df = pd.read_csv('../source_list/BH_data_clean_source_list.csv' , delimiter=',' , comment='#' ,)
df_bh = df.replace(' ' , '_' ,regex=True)
df_bh.insert(0  , 'type' , ['BH']*len(df_bh))
df = pd.read_csv('../source_list/NS_data_clean_source_list.csv' , delimiter=',' , comment='#' ,)
df_ns = df.replace(' ' , '_' ,regex=True)
df_ns.insert(0  , 'type' , ['NS']*len(df_ns))
df = pd.concat([df_bh , df_ns] , axis=0).reset_index(drop=True)
df 

,type,SRC_ID,A_NAME,A_RA,A_DEC,A_DATABASE_TABLE,A_OBSERVATORY,B_NAME,B_RA,B_DEC,XOFFSET_
0,BH,BH0001,1A_0620-00___________,95.685000,-0.345833,INTREFCAT_______,INTEGRAL_____,2CXO_J062244.5-002044_,95.685571,-0.345644,0.036
1,BH,BH0002,1E_1740.7-2942_______,265.978333,-29.745278,INTREFCAT_______,INTEGRAL_____,2CXO_J174354.8-294443_,265.978511,-29.745287,0.009
2,BH,BH0003,GINGA_1354-645_______,209.541250,-64.734722,INTREFCAT_______,INTEGRAL_____,2CXO_J135809.6-644405_,209.540413,-64.734813,0.022
3,BH,BH0004,GRO_J1655-40_________,253.500417,-39.845833,INTREFCAT_______,INTEGRAL_____,2CXO_J165400.1-395044_,253.500574,-39.845791,0.008
4,BH,BH0005,GRS_1758-258_________,270.302917,-25.740556,INTREFCAT_______,INTEGRAL_____,2CXO_J180112.3-254436_,270.301629,-25.743379,0.183
...,...,...,...,...,...,...,...,...,...,...,...
112,NS,NS0080,4U1745-203___________,267.222663,-20.367235,XRBCAT__________,_____________,2CXO_J174853.1-202202_,267.221514,-20.367450,0.066
113,NS,NS0081,E1048.1-5937_________,162.532607,-59.887640,XRBCAT__________,_____________,2CXO_J105007.1-595321_,162.529818,-59.889314,0.131
114,NS,NS0082,E1724-307____________,261.888086,-30.802042,XRBCAT__________,_____________,2CXO_J172733.1-304807_,261.888321,-30.802216,0.016
115,NS,NS0083,E2259+587____________,345.282670,58.879437,XRBCAT__________,_____________,2CXO_J230108.2+585244_,345.284506,58.878949,0.064


# Download data

In [8]:
start = int(input('Start number : '))
end = int(input('End number : '))

#start = 0 
#end = 3

df_select = df.iloc[start:end].reset_index(drop=True)

ra = df_select['B_RA']
dec = df_select['B_DEC']
name = df_select['A_NAME']
num_obs = [] 
offset = []
for index in tqdm(range(len(df_select))):
    try :
        s = search_csc(
                        radunit="arcsec", 
                        columns=fields, 
                        bands="broad",
                        clobber="yes" ,
                        radius= 3 , 
                        pos= str(ra[index])+','+str(dec[index])  ,
                        outfile='temp.csv')
                    #sys('search_csc outfile=trial.csv radunit=arcsec columns="SOS,SOP,SOV , o.gti_obs m.flux_aper_b" bands=broad clobber=yes radius=1 pos="65.428058,32.907468"')
                    #print(s)
                    
        data = pd.read_csv('temp.csv', delimiter='\t' , comment='#')
        data = data[data['match_type']=='u          ']
        data = data[data['instrument']=='ACIS']
        data.index.name= 'index'
        #f_name_fits = data_fits['A_NAME']
        n = len(data)
        num_obs.append(n)
        if(n>0):
            print('Match found --' , len(data))
            data.to_csv('../data/all_data/lmxb_v2_all/'+name[index]+'.csv')
    except:
        print('Manual ispection needed :' , name[index])
        num_obs.append(-1)

 33%|███▎      | 1/3 [00:17<00:35, 17.75s/it]

Match found -- 3


 67%|██████▋   | 2/3 [00:19<00:08,  8.23s/it]

Match found -- 13


100%|██████████| 3/3 [00:20<00:00,  6.96s/it]

Match found -- 3


In [9]:
df_updated = df_select.copy()
df_updated.insert(10 , 'num_obs' , num_obs)

df_updated.to_csv('../source_list/lmxb_updated/'+str(start)+'-'+str(end)+'.csv')